# Strategy Backtesting Notebook

Test and compare different investment strategies using historical simulations.

**Features:**
- Compare all 5 strategies
- Visualize historical performance
- Calculate risk metrics
- Sensitivity analysis

In [ ]:
# Setup
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import sys
import os

# Add project to path
sys.path.insert(0, os.path.dirname(os.getcwd()))

# Import from project
try:
    from app.services.price_generator import generate_portfolio_history
    from app.data.strategies import STRATEGIES
    print("Successfully imported project modules")
except ImportError as e:
    print(f"Import error: {e}")
    print("Make sure you're running from the notebooks directory")

In [ ]:
# Configuration
INITIAL_VALUE = 100000
SIMULATION_DAYS = 252  # 1 trading year
NUM_SIMULATIONS = 100  # For Monte Carlo

# Strategy definitions
print("Available Strategies:")
print("=" * 50)
for strategy_id, strategy in STRATEGIES.items():
    print(f"\n{strategy['name']}")
    print(f"  Risk Level: {strategy['risk_level']}/5")
    print(f"  Expected Return: {strategy['expected_return']['min']}% - {strategy['expected_return']['max']}%")
    print(f"  Volatility: {strategy['volatility']:.4f}")
    print(f"  Drift: {strategy['drift']:.5f}")

## 1. Single Strategy Simulation

In [ ]:
def simulate_strategy(strategy_id, days=252, seed=None):
    """Simulate a strategy for given number of days."""
    history = generate_portfolio_history(
        initial_value=INITIAL_VALUE,
        strategy=strategy_id,
        num_days=days,
        seed=seed
    )
    df = pd.DataFrame(history)
    df['date'] = pd.to_datetime(df['date'])
    df['returns'] = df['value'].pct_change()
    return df

# Simulate balanced strategy
df_balanced = simulate_strategy('balanced', SIMULATION_DAYS, seed=42)

# Plot
fig = px.line(
    df_balanced, 
    x='date', 
    y='value',
    title='Balanced Strategy - 1 Year Simulation'
)
fig.add_hline(y=INITIAL_VALUE, line_dash="dash", line_color="gray", annotation_text="Initial Value")
fig.update_layout(yaxis_title='Portfolio Value ($)', xaxis_title='Date')
fig.show()

# Summary stats
final_value = df_balanced['value'].iloc[-1]
total_return = (final_value - INITIAL_VALUE) / INITIAL_VALUE * 100
max_value = df_balanced['value'].max()
min_value = df_balanced['value'].min()
volatility = df_balanced['returns'].std() * np.sqrt(252) * 100  # Annualized

print(f"\nSimulation Results:")
print(f"  Final Value:    ${final_value:,.2f}")
print(f"  Total Return:   {total_return:+.2f}%")
print(f"  Max Value:      ${max_value:,.2f}")
print(f"  Min Value:      ${min_value:,.2f}")
print(f"  Volatility:     {volatility:.2f}% (annualized)")

## 2. Strategy Comparison

In [ ]:
# Simulate all strategies with same seed for fair comparison
results = {}
seed = 42

for strategy_id in STRATEGIES.keys():
    results[strategy_id] = simulate_strategy(strategy_id, SIMULATION_DAYS, seed)

# Create comparison chart
fig = go.Figure()

colors = {
    'conservative': 'blue',
    'value': 'purple',
    'balanced': 'green',
    'growth': 'orange',
    'aggressive': 'red'
}

for strategy_id, df in results.items():
    fig.add_trace(go.Scatter(
        x=df['date'],
        y=df['value'],
        name=STRATEGIES[strategy_id]['name'],
        line=dict(color=colors[strategy_id])
    ))

fig.add_hline(y=INITIAL_VALUE, line_dash="dash", line_color="gray")
fig.update_layout(
    title='Strategy Comparison - 1 Year Simulation',
    xaxis_title='Date',
    yaxis_title='Portfolio Value ($)',
    legend_title='Strategy'
)
fig.show()

In [ ]:
# Summary table
summary_data = []

for strategy_id, df in results.items():
    strategy = STRATEGIES[strategy_id]
    final_value = df['value'].iloc[-1]
    total_return = (final_value - INITIAL_VALUE) / INITIAL_VALUE * 100
    volatility = df['returns'].std() * np.sqrt(252) * 100
    max_drawdown = ((df['value'].cummax() - df['value']) / df['value'].cummax()).max() * 100
    sharpe = (df['returns'].mean() * 252) / (df['returns'].std() * np.sqrt(252)) if df['returns'].std() > 0 else 0
    
    summary_data.append({
        'Strategy': strategy['name'],
        'Risk Level': strategy['risk_level'],
        'Final Value': final_value,
        'Return (%)': total_return,
        'Volatility (%)': volatility,
        'Max Drawdown (%)': max_drawdown,
        'Sharpe Ratio': sharpe
    })

df_summary = pd.DataFrame(summary_data)
df_summary = df_summary.sort_values('Risk Level')

display(df_summary.style.format({
    'Final Value': '${:,.2f}',
    'Return (%)': '{:+.2f}%',
    'Volatility (%)': '{:.2f}%',
    'Max Drawdown (%)': '{:.2f}%',
    'Sharpe Ratio': '{:.2f}'
}))

## 3. Risk-Return Analysis

In [ ]:
# Risk-Return scatter plot
fig = px.scatter(
    df_summary,
    x='Volatility (%)',
    y='Return (%)',
    size='Risk Level',
    color='Strategy',
    title='Risk-Return Profile by Strategy',
    hover_data=['Sharpe Ratio', 'Max Drawdown (%)']
)

fig.update_layout(
    xaxis_title='Risk (Volatility %)',
    yaxis_title='Return (%)'
)
fig.show()

In [ ]:
# Sharpe ratio comparison
fig = px.bar(
    df_summary.sort_values('Sharpe Ratio', ascending=True),
    x='Sharpe Ratio',
    y='Strategy',
    orientation='h',
    title='Risk-Adjusted Returns (Sharpe Ratio)',
    color='Sharpe Ratio',
    color_continuous_scale='RdYlGn'
)
fig.show()

## 4. Monte Carlo Simulation

In [ ]:
def monte_carlo_simulation(strategy_id, days=252, n_simulations=100):
    """Run multiple simulations with different seeds."""
    all_simulations = []
    
    for i in range(n_simulations):
        df = simulate_strategy(strategy_id, days, seed=i)
        df['simulation'] = i
        all_simulations.append(df)
    
    return pd.concat(all_simulations, ignore_index=True)

# Run Monte Carlo for balanced strategy
print(f"Running {NUM_SIMULATIONS} simulations for Balanced strategy...")
mc_results = monte_carlo_simulation('balanced', SIMULATION_DAYS, NUM_SIMULATIONS)

# Calculate statistics at each time point
mc_stats = mc_results.groupby('date')['value'].agg(['mean', 'std', 'min', 'max', 
                                                     lambda x: x.quantile(0.05),
                                                     lambda x: x.quantile(0.95)])
mc_stats.columns = ['mean', 'std', 'min', 'max', 'p5', 'p95']
mc_stats = mc_stats.reset_index()

print(f"Simulations complete!")

In [ ]:
# Plot Monte Carlo results with confidence bands
fig = go.Figure()

# 90% confidence band
fig.add_trace(go.Scatter(
    x=mc_stats['date'],
    y=mc_stats['p95'],
    mode='lines',
    line=dict(width=0),
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=mc_stats['date'],
    y=mc_stats['p5'],
    mode='lines',
    line=dict(width=0),
    fill='tonexty',
    fillcolor='rgba(0, 100, 200, 0.2)',
    name='90% Confidence'
))

# Mean line
fig.add_trace(go.Scatter(
    x=mc_stats['date'],
    y=mc_stats['mean'],
    mode='lines',
    line=dict(color='blue', width=2),
    name='Mean'
))

# Initial value reference
fig.add_hline(y=INITIAL_VALUE, line_dash="dash", line_color="gray")

fig.update_layout(
    title=f'Monte Carlo Simulation - Balanced Strategy ({NUM_SIMULATIONS} runs)',
    xaxis_title='Date',
    yaxis_title='Portfolio Value ($)'
)
fig.show()

# Final value distribution
final_values = mc_results[mc_results['date'] == mc_results['date'].max()]['value']

print(f"\nFinal Value Distribution:")
print(f"  Mean:   ${final_values.mean():,.2f}")
print(f"  Std:    ${final_values.std():,.2f}")
print(f"  Min:    ${final_values.min():,.2f}")
print(f"  Max:    ${final_values.max():,.2f}")
print(f"  5th %:  ${final_values.quantile(0.05):,.2f}")
print(f"  95th %: ${final_values.quantile(0.95):,.2f}")

In [ ]:
# Histogram of final values
fig = px.histogram(
    final_values,
    nbins=30,
    title='Distribution of Final Portfolio Values',
    labels={'value': 'Final Value ($)'}
)
fig.add_vline(x=INITIAL_VALUE, line_dash="dash", line_color="red", annotation_text="Initial")
fig.add_vline(x=final_values.mean(), line_dash="dash", line_color="green", annotation_text="Mean")
fig.show()

# Probability of profit
prob_profit = (final_values > INITIAL_VALUE).mean() * 100
print(f"\nProbability of Profit: {prob_profit:.1f}%")
print(f"Probability of Loss:   {100-prob_profit:.1f}%")

## 5. Strategy Parameter Sensitivity

In [ ]:
# Test sensitivity to volatility parameter
volatilities = [0.005, 0.01, 0.015, 0.02, 0.025, 0.03]
sensitivity_results = []

for vol in volatilities:
    # Temporarily modify strategy
    np.random.seed(42)
    values = [INITIAL_VALUE]
    
    for _ in range(SIMULATION_DAYS - 1):
        daily_return = 0.0003 + np.random.normal(0, 1) * vol
        values.append(values[-1] * (1 + daily_return))
    
    final_val = values[-1]
    returns = pd.Series(values).pct_change().dropna()
    
    sensitivity_results.append({
        'Volatility': vol,
        'Final Value': final_val,
        'Return (%)': (final_val - INITIAL_VALUE) / INITIAL_VALUE * 100,
        'Realized Vol (%)': returns.std() * np.sqrt(252) * 100
    })

df_sensitivity = pd.DataFrame(sensitivity_results)

# Plot
fig = make_subplots(rows=1, cols=2, subplot_titles=['Final Value vs Volatility', 'Return vs Volatility'])

fig.add_trace(
    go.Scatter(x=df_sensitivity['Volatility'], y=df_sensitivity['Final Value'], mode='lines+markers'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=df_sensitivity['Volatility'], y=df_sensitivity['Return (%)'], mode='lines+markers'),
    row=1, col=2
)

fig.update_layout(title='Sensitivity Analysis: Volatility Parameter', showlegend=False)
fig.show()

display(df_sensitivity)

## 6. Conclusions

In [ ]:
# Final summary and recommendations
print("=" * 60)
print("BACKTESTING CONCLUSIONS")
print("=" * 60)

# Find best strategies by different metrics
best_return = df_summary.loc[df_summary['Return (%)'].idxmax(), 'Strategy']
best_sharpe = df_summary.loc[df_summary['Sharpe Ratio'].idxmax(), 'Strategy']
lowest_drawdown = df_summary.loc[df_summary['Max Drawdown (%)'].idxmin(), 'Strategy']

print(f"\nBest Strategy by Return:        {best_return}")
print(f"Best Strategy by Sharpe Ratio:  {best_sharpe}")
print(f"Lowest Max Drawdown:            {lowest_drawdown}")

print("\n" + "-" * 60)
print("RECOMMENDATIONS BY INVESTOR TYPE:")
print("-" * 60)
print("\nRisk-Averse Investors:")
print("  - Conservative or Value strategies")
print("  - Lower volatility, steadier returns")
print("  - Lower max drawdown")

print("\nModerate Risk Tolerance:")
print("  - Balanced strategy")
print("  - Good risk-adjusted returns")
print("  - Diversified approach")

print("\nHigh Risk Tolerance:")
print("  - Growth or Aggressive strategies")
print("  - Higher potential returns")
print("  - Be prepared for larger drawdowns")

print("\n" + "=" * 60)
print("Note: Past simulated performance does not guarantee future results.")
print("These simulations use simplified models and should not be used")
print("as the sole basis for investment decisions.")
print("=" * 60)